In [1]:
cd ..

/gorani/gorani/backend/dataserver


In [2]:
# parameters
user_id = 10
dropout = 0.2
neuron = 64
outlie_time = 200
bucket = 'ml'

In [3]:
from gorani.tf import FEATURE_LEN, transform_paragraphs
from gorani.datadb import DataDB

data_db = DataDB()
ps = data_db.get_flipped_paragraphs(user_id)

x,y = transform_paragraphs(ps)
print(len(x), len(y))

22 22


In [4]:
import numpy as np
idx = np.where(y > outlie_time)
print(len(idx))
x = np.delete(x, idx, axis=0)
y= np.delete(y, idx, axis=0)
print(x.shape, y.shape)
print(x.dtype, y.dtype)

2
(22, 268, 44) (22, 1)
float32 float32


In [6]:
def shuffle_split_data(X, y):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, 70)

    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]

    return X_train, y_train, X_test, y_test
x_train, x_valid, y_train, y_valid = shuffle_split_data(x,y)
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)
print(x_train.dtype, y_train.dtype)
print(FEATURE_LEN)

(17, 268, 44) (17, 1) (5, 268, 44) (5, 1)
float32 float32
44


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from gorani.filedb import FileDB
from typing import List
from gorani.tfmodels import time_model
import time

with tf.Graph().as_default():
    with tf.Session() as sess:
        K.set_session(sess)
        model = time_model(FEATURE_LEN, neuron, dropout)
        model.compile(loss='mse', optimizer='adagrad')
        filepath = "/tmp/time-model.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        tensorboard = TensorBoard(log_dir='/tmp/logs/{}-{}-{}-time-model-{}'.format(user_id, neuron, dropout, time.time()), histogram_freq=1,
                                  write_graph=True, write_images=False)
      
        history = model.fit(x_train, y_train, epochs=50, batch_size=int(x_train.shape[0]/20+1), verbose=1, validation_data=(x_valid, y_valid), callbacks=[checkpoint, tensorboard])
        file_db = FileDB()
        file_db.client.fput_object('ml', '{}-time-model.hdf5'.format(user_id), '/tmp/time-model.hdf5')

Train on 17 samples, validate on 5 samples
Epoch 1/50
16/17 [===========================>..] - ETA: 0s - loss: 8.7284
Epoch 00001: val_loss improved from inf to 2.49525, saving model to out/10-time-model.hdf5
17/17 [==============================] - 4s 245ms/step - loss: 8.2513 - val_loss: 2.4952
Epoch 2/50
16/17 [===========================>..] - ETA: 0s - loss: 2.2789
Epoch 00002: val_loss improved from 2.49525 to 0.90695, saving model to out/10-time-model.hdf5
17/17 [==============================] - 3s 183ms/step - loss: 2.1761 - val_loss: 0.9069
Epoch 3/50
16/17 [===========================>..] - ETA: 0s - loss: 1.7699
Epoch 00003: val_loss did not improve from 0.90695
17/17 [==============================] - 3s 182ms/step - loss: 1.9055 - val_loss: 2.2632
Epoch 4/50
16/17 [===========================>..] - ETA: 0s - loss: 1.0959
Epoch 00004: val_loss did not improve from 0.90695
17/17 [==============================] - 3s 179ms/step - loss: 1.0356 - val_loss: 1.8349
Epoch 5/50
16